In [1]:
from openai import OpenAI

import json
with open("history_5.jsonl", 'r', encoding='utf-8') as f:
    datas = [json.loads(line) for line in f]

In [2]:
user_utterances = []
for data in datas:
    for key, value in data.items():
        if key.startswith("turn_") and isinstance(value, dict):
            user_utterances.append(value.get("user", ""))

In [3]:
len(user_utterances)

990

In [4]:
class LLM_call:
    def __init__(self, llm_model=None):
        self.llm_model = llm_model

    def call_llm(self, llm_input):
        raise NotImplementedError

    def __call__(self, llm_input):
        return self.call_llm(llm_input)
class OPENAI_call(LLM_call):
    def __init__(self, llm_model=None):
        super(OPENAI_call, self).__init__(llm_model)
        self.model = llm_model
        self.temperature = 0
        self.max_tokens = 512
        self.seed = 39
        self.client = OpenAI()


    # @retry(stop_max_attempt_number=10)
    def call_llm(self, llm_input):
        # try:
        # print("self.model:", self.model)
        output = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": llm_input}
            ],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            seed=self.seed
        )
        # print(output)

        return output.choices[0].message.content
        # except Exception as e:
        #     LOGGER.warning(f"Failed to call LLM model: {e}")
        #     return None

    def set_temperature(self, temperature):
        self.temperature = temperature

    def set_max_tokens(self, max_tokens):
        self.max_tokens = max_tokens

    def set_seed(self, seed: int):
        self.seed = seed

In [5]:
import os
# 设置环境变量（在程序运行时生效，但不会写入系统）
os.environ["OPENAI_API_KEY"] = "sk-G6XflAR04SvxaTl1QhmIl6HimueZ0ZPYFeEVD78gELMRqth5"
os.environ["OPENAI_BASE_URL"] = "https://api.chatanywhere.tech"

In [6]:
prompt = """你是一位对话推荐系统领域的专家。

你的任务是：根据用户的一段对话内容，判断用户在对话中使用了哪一种或哪几种推荐交互策略，并说明判断理由。

可选策略包括以下3类（每条策略后都附有使用时的典型场景说明）：


1. 基于推荐结果给出反馈（Feedback on Recommendation）：
   用户对系统已经给出的推荐内容表达正向或负向情感态度。
   示例：这部电影听起来不错，我喜欢爱情片。

2. 寻求推荐（Seek Recommendation）：
   用户主动请求系统进行推荐，通常是对话的起始行为。
   示例：你能推荐一些适合聚餐的餐馆吗？

3. 属性询问（Attribute Inquiry）：
   用户对已经推荐的项目表示出一定兴趣，但想进一步了解其某些属性（如导演、位置、评分等）。
   示例：这家餐厅在哪儿？营业时间是几点？

用户的对话为：{user_utt}
【输出要求】  
请你严格按照以下 JSON 格式输出内容：
{{
  "判断理由": "<请在此填写你的分析过程和判断依据>",
  "策略": ["<填写一个或多个策略名称，例如：寻求推荐>"]
}} 
每个策略之间用英文的逗号隔开。
"""

In [7]:
LLM_config_action = {
    "model": "gpt-4o-mini",
    "temperature": 0,
    "max_tokens": 9000,
    "seed": 39,
}

Judge = OPENAI_call(LLM_config_action["model"])
Judge.set_temperature(LLM_config_action["temperature"])
Judge.set_max_tokens(LLM_config_action["max_tokens"])
Judge.set_seed(LLM_config_action["seed"])

In [8]:
user_utterances[0]

'我想找附近的米粉面馆，有没有推荐的？'

In [9]:
print(Judge(prompt.format(user_utt="能不能给我推荐一些附近的米粉面馆呢？")))

{
  "判断理由": "用户在对话中主动请求系统进行推荐，询问附近的米粉面馆，这符合寻求推荐的策略。用户的提问表明他们希望系统提供相关的推荐信息。",
  "策略": ["寻求推荐"]
}


In [10]:
from concurrent.futures import ThreadPoolExecutor, as_completed

dict = {
    "寻求推荐": 0,
    "基于推荐结果给出反馈": 0,
    "偏好澄清": 0,
    "属性询问": 0
}

def process_utterance(utt):
    res = Judge(prompt.format(user_utt=utt))
    res = eval(res)
    return res

user_utterances = user_utterances  # 保证在有效范围内

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_utterance, utt) for utt in user_utterances]
    
    for future in as_completed(futures):
        try:
            res = future.result()
            print(res)
            actions = res['策略']
            for action in actions:
                dict[action] += 1
        except Exception as e:
            print(f"Error processing utterance: {e}")

print(dict)

{'判断理由': '用户询问饺子馆的电话，表明他们对该餐馆感兴趣，并希望获取更多具体信息。这种行为属于对已经推荐的项目（饺子馆）进行属性询问，因为用户想了解该餐馆的联系方式。', '策略': ['属性询问']}
{'判断理由': '用户在对话中询问便利便当的营业时间，表明他们对已经推荐的便利便当感兴趣，并希望了解其具体的属性信息，因此属于属性询问的策略。', '策略': ['属性询问']}
{'判断理由': '用户询问饺子馆的营业时间，表明他们对该餐馆感兴趣，并希望获取更多具体信息。这种行为符合属性询问的特征，因为用户在寻求关于已经推荐或已知的项目的具体属性信息。', '策略': ['属性询问']}
{'判断理由': '用户在对话中主动询问附近是否有好吃的米粉面馆，并且提供了具体的位置信息（武侯祠），这表明用户在寻求推荐。用户的提问方式符合寻求推荐的典型场景，因此可以判断出用户使用了这一策略。', '策略': ['寻求推荐']}
{'判断理由': '用户在对话中明确表示了想尝试豫菜，并询问是否有推荐的餐厅。这表明用户在主动请求系统进行推荐，属于寻求推荐的行为。', '策略': ['寻求推荐']}
{'判断理由': '用户表达了饥饿的情绪并主动请求推荐适合吃饺子和混沌的地方。这明确表明用户在寻求建议，因此属于寻求推荐策略的范畴。', '策略': ['寻求推荐']}
{'判断理由': '用户只提到自己所在的位置，但没有明确请求推荐或询问已推荐项目的属性，因此可以推测用户可能是在为后续的推荐做准备，但目前的对话中没有明确的寻求推荐或反馈。', '策略': []}
{'判断理由': '用户在对话中提到自己有点失望，并希望和朋友一起庆祝毕业，明确寻求附近好吃的韩料，因此表现出主动请求推荐。', '策略': ['寻求推荐']}
{'判断理由': '用户在对话中主动询问是否有饺子混沌店，这表明他们正在寻求一些相关的推荐。这种询问显然是在请求系统提供特定类型的餐馆信息，因此可归类为寻求推荐策略。', '策略': ['寻求推荐']}
{'判断理由': '用户已经对米粉面馆表示出一定的兴趣，并且主动询问了营业时间，这表明用户在寻求进一步的信息来安排就餐。因此，用户的行为符合属性询问的特征。', '策略': ['属性询问']}
{'判断理由': '用户在对话中询问推荐的米粉面馆的特色菜品

In [15]:
import numpy as np
from scipy.spatial.distance import cosine, euclidean, cityblock

# 计算余弦相似度
def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

# 计算欧氏距离相似度
def euclidean_similarity(vec1, vec2):
    return 1 / (1 + euclidean(vec1, vec2))

# 计算曼哈顿距离相似度
def manhattan_similarity(vec1, vec2):
    return 1 / (1 + cityblock(vec1, vec2))

# 计算皮尔逊相关系数相似度
def pearson_similarity(vec1, vec2):
    return np.corrcoef(vec1, vec2)[0, 1]

# 计算杰卡德相似度
def jaccard_similarity(vec1, vec2):
    intersection = np.sum(np.minimum(vec1, vec2))
    union = np.sum(np.maximum(vec1, vec2))
    return intersection / union

# 计算余弦相似度测试
vec1 = np.array([475/990, 161/990, 504/990])
vec2 = np.array([1239/1988, 716/1988, 453/1988])

print(f"余弦相似度: {cosine_similarity(vec1, vec2)}")
print(f"欧氏距离相似度: {euclidean_similarity(vec1, vec2)}")
print(f"曼哈顿距离相似度: {manhattan_similarity(vec1, vec2)}")
print(f"皮尔逊相关系数相似度: {pearson_similarity(vec1, vec2)}")
print(f"杰卡德相似度: {jaccard_similarity(vec1, vec2)}")


余弦相似度: 0.8733777161591179
欧氏距离相似度: 0.7286502374980394
曼哈顿距离相似度: 0.6164468516145036
皮尔逊相关系数相似度: 0.11216771134753713
杰卡德相似度: 0.5831132408842916
